In [4]:
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
input("Please copy 'depth_data' into current path!")
file_name = 'depth_data'
with open(file_name, 'rb') as f:
    save = pickle.load(f)
    dataset = save['dataset']
    names = save['names']
    orientations = save['orientations']
    del save
#print(dataset.shape)
image1 = dataset[0,:,:]
plt.imshow(image1,cmap='Greys_r')
plt.show()

Please copy 'depth_data' into current path!
